# [EEP 118] Introductory Applied Econometrics, Fall 2023

# Problem Set 2

* Instructor: Aprajit Mahajan
* GSIs: Abdoulaye Cisse and Shuo Yu

**About This Notebook**
* This notebook is meant to guide you to answer Questions 5(e)-5(h) and Question 6 in the homework. You can use it to run the codes needed to answers questions in these sections of the homework. Once you run these codes, you can rely on the output generated to write your answers in the same PDF file where you answer the other questions. Do not put your answers directly in this notebook. Please submit all your answers in one PDF file on Gradescope.

## Setup

In [ ]:
import statsmodels.api as sm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.iolib.summary2 import summary_col
from scipy.stats import norm

## Question 5
### Q5(d)

The Current Population Survey (CPS) is the main source of a range of 
US labor force statistics including employment, hours worked and 
earnings. If you are interested in writing an honors thesis on labor 
force issues in the US this is a great source of data. See 
[http://www.bls.gov/cps/](http://www.bls.gov/cps/) for more details. The data set 
`cps92_08.dta` from bcourses which contains information on average 
hourly earnings, gender and college status for a sample of American
workers.

In [ ]:
# Load the dataset
data = pd.read_stata('cps92_08.dta')

In [ ]:
# Check the first 5 rows of the data frame to get a sense of what your data looks like.
data.head(5)

First, estimate the model
\begin{equation*}
 Y=\beta _{0}+\beta _{1}X_{1}+\beta _{2}X_{2}+u
\end{equation*}
where the dependent variable is average hourly
earnings ($Y=$ `ahe`), $X_{1}$ denotes gender ($X_{1}=$ `female`) and
is equal to 1 if the worker is female and $X_{2}$ is a crude measure
of education and is equal to 1 if the worker has a college degree 
($X_{2}=$`bachelor`}).

Regress `ahe` on `female` and `bachelor`.

In [ ]:
# regress `ahe` on `female` and `bachelor`
X = sm.add_constant(data[['female','bachelor']])
y = data['ahe']
model_ahe_bachelor_female = sm.OLS(y, X).fit(cov_type='HC1')
print(model_ahe_bachelor_female.summary())

### Q5(e)

 Next, regress `female` on a constant and the variable `bachelor`.
 Then, obtain an estimate of $\epsilon$ where 
 \begin{equation*}
  \epsilon \equiv X_{1}-\delta _{0}-\delta _{1}X_{2} 
 \end{equation*}
 and 
 \begin{equation*} 
 \delta _{1}\equiv \frac{Cov\left(X_{1},X_{2}\right) }{Var\left( X_{2}\right) }
 \qquad \text{and } \ \delta_{0}\equiv \mathbb{E}\left( X_{1}\right) 
 -\delta_{1}\mathbb{E}\left( X_{2}\right) 
 \end{equation*} 
The command  `y-model.predict(X)` produces (for observation $i$) 
\begin{align*}
 \mathtt{epsilonhat}_{i} = 
 \mathtt{female}_{i}-\hat{\delta}_{0}-\hat{\delta}_{1}\mathtt{bachelor}_{i}
\end{align*} 
where $(\hat{\delta}_{0},\hat{\delta}_{1})$ are the OLS coefficients from 
the regression of `female` on a constant and  `bachelor`.

In [ ]:
X = sm.add_constant(data['bachelor'])
y = data['female']
model = sm.OLS(y, X).fit(cov_type='HC1')
epsilonhat = y - model.predict(X)
data['epsilonhat'] = epsilonhat

### Q5(f)

 Regress `epsilonhat` on a constant and `bachelor`. The coefficient 
 on `bachelor` is zero (to machine accuracy). Is this what you would 
 expect? Why or why not?

In [ ]:
X = sm.add_constant(data['bachelor'])
model_epsilon = sm.OLS(data['epsilonhat'], X).fit(cov_type='HC1')
print(model_epsilon.summary())

### Q5(g)

Regress `ahe` on `epsilonhat`.

In [ ]:
X = sm.add_constant(data['epsilonhat'])
y = data['ahe']
model_ahe_epsilon = sm.OLS(y, X).fit(cov_type='HC1')
print(model_ahe_epsilon.summary())

### Q5(h)

Regress `ahe` on `female` and `bachelor`. Compare the coefficient on
`female` from this regression to the coefficient on `epsilonhat` from
the previous regression. What do you conclude from the observation
that they are identical?

In [ ]:
X = sm.add_constant(data[['female', 'bachelor']])
y = data['ahe']
model_ahe_female_bachelor = sm.OLS(y, X).fit(cov_type='HC1')
print(model_ahe_female_bachelor.summary())

## Question  6

### Q6(a)

Using the Jupyter notebook code, estimate this model where
  $y=$ `ahe`, $X_{1}=$ `female` and $X_{2}=$ `bachelor`.
  Then, run the relevant cells in the notebook to
  obtain predictions for the average hourly wage
  for all four possible combinations of `female` and
  `bachelor`.

In [ ]:
# Regress `ahe` on `female`, `bachelor` and a constant.
X = data[['female', 'bachelor']]
X = sm.add_constant(X)  # Adds a constant column for the intercept term
y = data['ahe']
# Estimate the model
model = sm.OLS(y, X).fit(cov_type='HC1')
# Display the model summary
print(model.summary())

In [ ]:
## Obtaining Predictions
# Obtain Predictions
combinations = [
    [1, 0, 0],  # constant, female=0, bachelor=0
    [1, 0, 1],  # constant, female=0, bachelor=1
    [1, 1, 0],  # constant, female=1, bachelor=0
    [1, 1, 1],  # constant, female=1, bachelor=1
]
predictions = model.predict(combinations)

### Q6(b)

Construct confidence intervals for each of the four predictions
  from the linear regression model.

In [ ]:
# Compute standard errors of predictions
X_combinations = np.array(combinations)
cov_matrix = model.cov_params()
se_predictions = [np.sqrt(X_combinations[i].dot(cov_matrix).dot(X_combinations[i].T)) for i in range(len(X_combinations))]

This is accomplished in the Python
  notebook by the commands (the `norm` function is imported
  from `scipy.stats` in the Setup Section of the notebook):

In [ ]:
# Compute the 95% confidence intervals for predictions using the normal approximation
confidence_level = 0.95
z_value = norm.ppf(1 - (confidence_level/2))
conf_intervals = [(predictions[i] - z_value*se_predictions[i], predictions[i] + z_value*se_predictions[i]) for i in range(len(predictions))]

In [ ]:
# Display predictions and confidence intervals
print("\nPredictions with Confidence Intervals:")
combo_labels = [
    "Male without a Bachelor's Degree",
    "Male with a Bachelor's Degree",
    "Female without a Bachelor's Degree",
    "Female with a Bachelor's Degree"
]
for label, pred, (lower, upper) in zip(combo_labels, predictions, conf_intervals):
    print(f"\nFor {label}:")
    print(f"Predicted: ${pred:.5f}")
    print(f"95% CI: [${lower:.5f}, ${upper:.5f}]")

### Q6(c)

Next, we will compare these predictions
  to the actual sample conditional means. First, we compute the four
  different conditional means -- i.e. average wages for each of the
  possible values that `female` and `bachelor`
  variables take together (i.e. (0,0),(1,0),(0,1) and
  (1,1)). Sometimes such cell means are known as "cross-tabs." This
  is accomplished in the code by:

In [ ]:
# Compute conditional statistics for the dataset
grouped = data.groupby(['female', 'bachelor']).agg({
    'ahe': ['mean', 'std', 'count']
}).reset_index()

Run the python cell that computes these four cell means. Compare
  the predictions in 6(a) to these conditional means by
  running the corresponding cells in Python. Are they identical?

In [ ]:
# Print sub-sample means, predictions and their difference

print("\nSub-sample Statistics, Predictions, and Differences:")
for index, row in grouped.iterrows():
    gender_value = row[('female', '')]  # Access the 'female' value
    bachelor_value = row[('bachelor', '')]  # Access the 'bachelor' value
    
    gender = "Female" if gender_value == 1 else "Male"
    education = "with a Bachelor's Degree" if bachelor_value == 1 else "without a Bachelor's Degree"
    
    mean = row[('ahe', 'mean')]
    std = row[('ahe', 'std')]
    count = row[('ahe', 'count')]
    
    prediction = predictions[index]
    difference = prediction - mean
    
    print(f"\nFor {gender} {education}:")
    print(f"Actual Mean: ${mean:.5f}")
    print(f"Predicted: ${prediction:.5f}")
    print(f"Difference: ${difference:.5f}")
    print(f"Standard Deviation: {std:.5f}")
    print(f"Sample Size: {int(count)}")

### Q6(e)

(**Adding an interaction term**) Next, use the relevant Python cell to
  estimate the regression coefficients in the following regression
\begin{equation*} 
\texttt{ahe}=\delta _{0}+\delta
_{1}\texttt{female}+\delta _{2}
\texttt{bachelor}+\delta _{3}\texttt{bachelor*female}+v
\end{equation*}
where $\mathbb{E}\left( v\mathbf{x}\right) =0$ and
$\mathbf{x=}\left( 1,\texttt{female},\texttt{bachelor},\texttt{bachelor*female}\right) ^{\prime}$.
Run the relevant Python cell that compares
the predicted sample means from this model to the actual conditional
sample means (the "cross-tabs").  How different are they?  What
does this tell you about the relationships between the cross-tabs
constructed in 6(c) above and the regression carried
out here.

In [ ]:
# 1. Create interaction term
data['bachelor_female'] = data['bachelor'] * data['female']

# 2. Construct the regression model
X = data[['female', 'bachelor', 'bachelor_female']]
X = sm.add_constant(X)  # Add a constant to represent the intercept
y = data['ahe']

# 3. Estimate the model
model = sm.OLS(y, X).fit(cov_type='HC1')

# 4. Display the results
print(model.summary())

# Compute the predicted means from this model: 
# 1. Obtain Predictions
combinations = [
    [1, 0, 0, 0],  # constant, female=0, bachelor=0, bachelor_female=0
    [1, 0, 1, 0],  # constant, female=0, bachelor=1, bachelor_female=0
    [1, 1, 0, 0],  # constant, female=1, bachelor=0, bachelor_female=0
    [1, 1, 1, 1],  # constant, female=1, bachelor=1, bachelor_female=1
]
predictions = model.predict(combinations)

# Compare predictions to conditional Means 
grouped = data.groupby(['female', 'bachelor']).agg({
    'ahe': ['mean', 'std', 'count']
}).reset_index()

# Give names to the columns for easier referencing
grouped.columns = ['female', 'bachelor', 'ahe_mean', 'ahe_std', 'ahe_count']

# Using the regression equation & predictions (assuming model is already fit as in prior code)
combinations = [
    [1, 0, 0, 0],  # constant, female=0, bachelor=0, bachelor_female=0
    [1, 0, 1, 0],  # constant, female=0, bachelor=1, bachelor_female=0
    [1, 1, 0, 0],  # constant, female=1, bachelor=0, bachelor_female=0
    [1, 1, 1, 1],  # constant, female=1, bachelor=1, bachelor_female=1
]
predictions = model.predict(combinations)

# Compare the predictions to the conditional means
conditional_means = grouped['ahe_mean'].values

# Construct and print the comparison table
output = []
output.append("{:<25} {:<20} {:<20} {:<20}".format("Group (female, bachelor)", 
                                                   "Predicted Value", 
                                                   "Conditional Mean", 
                                                   "Difference"))
output.append("-" * 85)
for i, combination in enumerate(combinations):
    difference = predictions[i] - conditional_means[i]
    output.append("{:<25} {:<20.5f} {:<20.5f} {:<20.5f}".format(str(combination[1:3]), 
                                                                predictions[i], 
                                                                conditional_means[i], 
                                                                difference))
print("\n".join(output))

### Q6(f)

(**Adding age as a regressor**) Now, estimate the mean of $\texttt{ahe}$ conditional on 
$\texttt{age}$, $\texttt{female}$, and $\texttt{bachelor}$ by running the relevant Python
  cell.  The output reports the sample means for ages 28, 30 and 32
  for each of the four categories defined by $\texttt{female}$ and
  $\texttt{bachelor}$ (i.e. for values $\left( 0,1\right) ,$
  $\left( 1,0\right) ,$ $\left( 0,0\right) $ and $\left( 1,1\right) $
  respectively). How many means are estimated?

In [ ]:
# Group by 'age', 'female', and 'bachelor' and compute the aggregation functions
grouped = data.groupby(['age', 'female', 'bachelor']).agg({
    'ahe': ['mean', 'std', 'count']
}).reset_index()

# Rename columns for easier referencing
grouped.columns = ['Age', 'Female', 'Bachelor', 'Mean of AHE', 'Std Dev of AHE', 'Count of AHE']

# Filter the data to only include ages 28, 30, and 32
filtered_data = grouped[grouped['Age'].isin([28, 30, 32])]

# Pretty Print using Pandas Styling
def highlight(s):
    return ['background-color: lightgrey' if s.name in ["Age", "Female", "Bachelor"] else '' for v in s]

styled = filtered_data.style.apply(highlight, axis=1)\
    .set_properties(**{'text-align': 'center'})\
    .hide_index()\
    .highlight_max(subset=["Mean of AHE", "Std Dev of AHE", "Count of AHE"], color='lightgreen')\
    .highlight_min(subset=["Mean of AHE", "Std Dev of AHE", "Count of AHE"], color='salmon')

print(filtered_data)

### Q6(g)

Next, run the Python cell that estimates the regression specification
\begin{equation*}
\texttt{ahe}=\delta _{0}+\delta _{1}\texttt{female}+\delta _{2}
\texttt{bachelor}+\delta _{3}\texttt{age}+v
\end{equation*}
where $\mathbb{E}\left( v\mathbf{x}\right) =0$ and 
$\mathbf{x=}\left(1,\texttt{female},\texttt{bachelor},\texttt{age}\right)^{\prime}$. 
Compare the predicted sample means in this model for the
age-sex-bachelor categories you calculated in part (e) above.  How
different are they?  Comment on the advantages and disadvantages of
learning about sub-sample means using multiple regression (versus the
cross-tab means carried out in part (e)).

In [ ]:
# Comparing the predictions from OLS to sub-group means when we have a
# regressor that takes on multiple values (`age`)

# Create LHS and RHS data 
X = data[['female', 'bachelor', 'age']]
X = sm.add_constant(X)  # Adds a constant column for the intercept term
y = data['ahe']

# Estimate Model 
model = sm.OLS(y, X).fit(cov_type='HC1')

# Print model summary 
print(model.summary())

# Predict Sample Means for the specific age-sex-bachelor categories
ages = [28, 30, 32]
combinations = [
    [1, 0, 0, age] for age in ages
] + [
    [1, 0, 1, age] for age in ages
] + [
    [1, 1, 0, age] for age in ages
] + [
    [1, 1, 1, age] for age in ages
]

predictions = model.predict(combinations)

# Prepare data for comparison
# Filtering for specific ages and aggregating by 'female', 'bachelor', and 'age'
filtered_data = data[data['age'].isin([28, 30, 32])]
grouped = filtered_data.groupby(['female', 'bachelor', 'age']).agg({
    'ahe': ['mean', 'std', 'count']
}).reset_index()
grouped.columns = ['Female', 'Bachelor', 'Age', 'Mean of AHE', 'Std Dev of AHE', 'Count']

# Add the predictions to our grouped data
grouped['Predicted AHE'] = predictions

# Calculate the difference between conditional mean and predicted mean
grouped['Difference'] = grouped['Mean of AHE'] - grouped['Predicted AHE']

# Displaying the results
cols_to_display = ['Age', 'Female', 'Bachelor', 'Mean of AHE', 'Predicted AHE', 'Difference']
print(grouped[cols_to_display])